In [121]:
import pandas as pd
import numpy as np
import os

In [122]:
df_panel1 = pd.read_stata(os.path.join('DATA', 'df_panel1.dta')) 

In [123]:
df_panel1 = df_panel1[~df_panel1['sales_log'].isna()]
df_panel1 = df_panel1[df_panel1['sales_log']!=0]

In [124]:
import numpy as np
from sklearn.linear_model import LinearRegression

def get_sales_r(df):
    X = df[['t_log', 'v9', 'v10', 'v11', 'v12']]
    y = df['sales_log']
    reg = LinearRegression().fit(X, y)
    df['sales_log_hat'] = reg.predict(X)
    df['sales_r'] = df['sales_log'] - df['sales_log_hat']

    return df

In [125]:
%%time
df = pd.DataFrame()
for i in df_panel1['gvkey_hscode6'].unique():
    df_ = df_panel1[df_panel1['gvkey_hscode6']==i].sort_values(by=['t'])
    df = df.append(get_sales_r(df_))

Wall time: 2min 48s


In [126]:
df_panel1 = df.copy()

In [127]:
df_panel1

,v1,gvkey_hscode6,vol,code2,month,quarter,t,v9,v10,v11,...,sigmar_median,sigmar_norm,sales_median,sales_norm,gm_median,gm_norm,inv_median,inv_norm,sales_log_hat,sales_r
3,3,10000_850440,2.16,85,1,1,4,1,0,0,...,0.373878,0.422971,244.005005,0.735030,0.303814,0.864445,182.177994,0.765575,5.222932,-0.033587
4,4,10000_850440,2.19,85,4,2,5,0,1,0,...,0.373878,0.832540,244.005005,0.946899,0.303814,0.880070,182.177994,0.780391,5.413282,0.029343
5,5,10000_850440,1.18,85,7,3,6,0,0,1,...,0.373878,1.953349,244.005005,0.932522,0.303814,0.916696,182.177994,0.841358,5.409070,0.018256
6,6,10000_850440,4.00,85,10,4,7,0,0,0,...,0.373878,0.987323,244.005005,0.708883,0.303814,0.931077,182.177994,0.894098,5.172190,-0.019066
7,7,10000_850440,3.16,85,1,1,8,1,0,0,...,0.373878,1.429949,244.005005,0.902563,0.303814,0.845493,182.177994,0.871571,5.320775,0.073897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188394,188394,9899_854442,20.84,85,10,4,39,0,0,0,...,0.945029,1.522442,33089.000000,1.451902,0.553235,0.930564,NaN,NaN,10.656759,0.123072
188395,188395,9899_854442,11.16,85,1,1,40,1,0,0,...,0.945029,2.332663,33089.000000,1.356010,0.553235,0.972079,NaN,NaN,10.613190,0.098312
188396,188396,9899_854442,4.28,85,4,2,41,0,1,0,...,0.945029,3.124470,33089.000000,1.359576,0.553235,0.983391,NaN,NaN,10.614278,0.099851
188397,188397,9899_854442,5.69,85,7,3,42,0,0,1,...,0.945029,3.118460,33089.000000,1.347517,0.553235,0.990529,NaN,NaN,10.643749,0.061471


In [128]:
df_panel1.rename(columns={"t": "period"}, inplace=True)

In [129]:
df_panel1.columns

Index(['v1', 'gvkey_hscode6', 'vol', 'code2', 'month', 'quarter', 'period',
       'v9', 'v10', 'v11', 'v12', 'vol_log', 't_log', 'vol_log_hat', 'r',
       'sigmar', 'source1', 'n', 'v20', 'despues', 'antes', 'consistency_8',
       'consistency_china', 'gvkey', 'hscode6', 'year', 'date', 'ap', 'cogs',
       'icap', 'inv', 'invtq', 'ppent', 'sales', 'mkval', 'dap', 'invr', 'gm',
       'tend', 'sales_log', 'id', 'dap_median', 'dap_norm', 'sigmar_median',
       'sigmar_norm', 'sales_median', 'sales_norm', 'gm_median', 'gm_norm',
       'inv_median', 'inv_norm', 'sales_log_hat', 'sales_r'],
      dtype='object')

In [130]:
df_panel1.rename(columns={"consistency_8": "t"}, inplace=True)

In [131]:
df_panel1.to_csv(os.path.join('DATA', 'df_panel1_.csv'))

In [134]:
def get_N():
    df = df_panel1[df_panel1['t']==1][['gvkey_hscode6', 'source1']].groupby(['gvkey_hscode6']).nunique()
    df.reset_index(inplace=True)
    df.rename(columns={'source1': 'N'}, inplace=True)
    dictio = {gvhs: n for gvhs, n in zip(df['gvkey_hscode6'], df['N'])}
    
    N = dict()
    N[1] = list(df['gvkey_hscode6'][df['N']==1].unique())
    N[2] = list(df['gvkey_hscode6'][df['N']==2].unique())
    N[3] = list(df['gvkey_hscode6'][df['N']==3].unique())
    N[0] = list(set(df_panel1['gvkey_hscode6'].unique()).difference(set(N[1])).difference(set(N[2])).difference(set(N[3])))

    return N

In [135]:
N = get_N()

In [136]:
# Obs. con 2 sources distintas
lista = []
for gvkey in N[2]:
    df = df_panel1[df_panel1['gvkey_hscode6']==gvkey].copy()
    sources = list(df[df['t']==1]['source1'].unique())
    if len(sources)==1: continue
    lista.append(gvkey)

In [137]:
len(N[2])

230

In [138]:
len(lista)

230

In [139]:
df_panel2 = df_panel1[np.logical_and(df_panel1['gvkey_hscode6'].isin(N[2]), df_panel1['t']==1)].copy()
df_panel2

,v1,gvkey_hscode6,vol,code2,month,quarter,period,v9,v10,v11,...,sigmar_median,sigmar_norm,sales_median,sales_norm,gm_median,gm_norm,inv_median,inv_norm,sales_log_hat,sales_r
548,548,100095_611610,17.000000,61,10,4,7,0,0,0,...,0.448129,4.106791,4628.308594,0.848798,0.071672,1.007601,1134.921387,0.653696,8.244570,0.031443
550,550,100095_611610,35.820000,61,4,2,9,0,1,0,...,0.448129,4.314202,4628.308594,0.847731,0.071672,0.949592,1134.921387,NaN,8.265714,0.009041
552,552,100095_611610,40.810001,61,10,4,11,0,0,0,...,0.448129,0.250201,4628.308594,0.867502,0.071672,1.031937,1134.921387,0.723650,8.348459,-0.050650
554,554,100095_611610,56.490002,61,4,2,13,0,1,0,...,0.448129,0.690211,4628.308594,0.885312,0.071672,0.934726,1134.921387,NaN,8.351926,-0.033794
556,556,100095_611610,91.080002,61,10,4,15,0,0,0,...,0.448129,1.752765,4628.308594,0.997694,0.071672,0.998938,1134.921387,0.841957,8.422170,0.015468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188393,188393,9899_854442,5.610000,85,7,3,38,0,0,1,...,0.945029,1.284157,33089.000000,1.382302,0.553235,0.989275,NaN,NaN,10.624080,0.106627
188394,188394,9899_854442,20.840000,85,10,4,39,0,0,0,...,0.945029,1.522442,33089.000000,1.451902,0.553235,0.930564,NaN,NaN,10.656759,0.123072
188395,188395,9899_854442,11.160000,85,1,1,40,1,0,0,...,0.945029,2.332663,33089.000000,1.356010,0.553235,0.972079,NaN,NaN,10.613190,0.098312
188396,188396,9899_854442,4.280000,85,4,2,41,0,1,0,...,0.945029,3.124470,33089.000000,1.359576,0.553235,0.983391,NaN,NaN,10.614278,0.099851


In [140]:
df = df_panel2[['gvkey_hscode6', 'source1', 'date']].groupby(['gvkey_hscode6', 'source1']).first()
df.reset_index(inplace=True)
df_source1 = df[['gvkey_hscode6', 'source1']].copy()
df_source1

,gvkey_hscode6,source1
0,100095_611610,Bangladesh
1,100095_611610,China
2,10016_854442,Brazil
3,10016_854442,China
4,100609_850131,China
...,...,...
455,9016_850300,China
456,9818_851830,China
457,9818_851830,Hong Kong
458,9899_854442,China


In [141]:
from datetime import datetime
from dateutil import relativedelta
dictio = dict()
for gv_hs in df_source1['gvkey_hscode6'].unique():
    country = list(df_source1[df_source1['gvkey_hscode6']==gv_hs]['source1'])
    dates1 = list(df_panel2[np.logical_and(df_panel2['gvkey_hscode6']==gv_hs, df_panel2['source1']==country[0])]['date'].copy())
    dates2 = list(df_panel2[np.logical_and(df_panel2['gvkey_hscode6']==gv_hs, df_panel2['source1']==country[1])]['date'].copy())
    if dates1[-1] < dates2[0]: dictio[gv_hs] = [country[0], country[1]]
    if dates1[0] > dates2[-1]: dictio[gv_hs] = [country[1], country[0]]

In [142]:
df = pd.DataFrame()
df['gvkey_hscode6'] = dictio.keys()
df['source1'] = [lista[0] for lista in dictio.values()]
df['source2'] = [lista[1] for lista in dictio.values()]
df

,gvkey_hscode6,source1,source2
0,100095_611610,China,Bangladesh
1,10016_854442,Brazil,China
2,100609_850131,China,Czech Republic
3,100609_853890,China,Germany
4,100609_854430,China,Austria
...,...,...,...
213,8551_870899,China,Hong Kong
214,8823_940360,China,Austria
215,9016_850300,China,Brazil
216,9818_851830,Hong Kong,China


In [143]:
df_panel2['T1'] = 0
for index, row in df.iterrows():
    df_panel2.loc[np.logical_and(df_panel2["gvkey_hscode6"]==row['gvkey_hscode6'], df_panel2["source1"]==row['source1']), 'T1'] = 1

df_panel2['T2'] = 0
for index, row in df.iterrows():
    df_panel2.loc[np.logical_and(df_panel2["gvkey_hscode6"]==row['gvkey_hscode6'], df_panel2["source1"]==row['source2']), 'T2'] = 1


In [144]:
df_panel2['china_out'] = 0
df_panel2.loc[df_panel2["gvkey_hscode6"].isin(df[df['source1']=='China']['gvkey_hscode6']), 'china_out'] = 1

df_panel2['china_inn'] = 0
df_panel2.loc[df_panel2["gvkey_hscode6"].isin(df[df['source2']=='China']['gvkey_hscode6']), 'china_inn'] = 1

In [145]:
df_panel2.to_csv(os.path.join('DATA', 'df_panel2_.csv'))